In [2]:
!rm -rf kakao
!mkdir -p kakao/static
!mkdir -p kakao/templates
!touch kakao/app.py
!touch kakao/templates/webservice.html
!tree kakao

kakao
├── app.py
├── static
└── templates
    └── webservice.html

2 directories, 2 files


## app.py 작성

In [12]:
%%writefile kakao/app.py
# Data Processing
import pandas as pd

# Machine Learning
from sklearn.metrics.pairwise import cosine_similarity 

# mongodb
import pymongo

import flask
from flask import *

app = flask.Flask(__name__)
app.config["DEBUG"] = True

from flask_cors import CORS
CORS(app)

# main index page route
@app.route('/')
def home():
    return '<h1>API is working.. </h1>'

@app.route('/main')
def main():
    return render_template("webservice.html")

@app.route('/predict',methods=['GET'])
def predict():

    def find_wine(question1_answer, question2_answer, question3_answer, question4_answer, question5_answer, result):
        path = "/home/ubuntu/python3/notebook/workspace/project/Wine_Recommendation_Modeling/kakao/static/data"
        wine_data = pd.read_csv('{}/wine_data.csv'.format(path))
        wine_data.drop('Unnamed: 0', axis=1, inplace=True)
        customer_data = pd.read_csv('{}/customer_data.csv'.format(path))
        customer_data.drop('Unnamed: 0', axis=1, inplace=True)

        # 1. 설문조사 만들기
        question1 = {0 : 'black', 1 : 'milk_sweet', 2: 'tea'}
        question2 = {0 : 'acid', 1 : 'body'}
        question3 = {0 : 'meat', 1 : 'cheese', 2: 'seafood', 3 : 'vegetables', 4 : 'dessert'}
        question5 = 0
        # 중복 선택 가능
        question4 = {0 : 'floral', 1 : 'fruit', 2 : 'citrus', 3 : 'oriental_spice', 4 : 'oriental_leather', 5: 'earth'} 
        questions = [question1, question2, question3, question4, question5]

        # 2. 설문조사 답에 따른 와인 특성 값
        question1_result = list(question1.values())[question1_answer]
        question2_result = list(question2.values())[question2_answer]
        question3_result = list(question3.values())[question3_answer]
        question4_result = [list(question4.values())[int(i)] if question4_answer >= 10 else list(question4.values())[question4_answer]
                    for i in str(question4_answer)]
        question5_result = question5_answer

        # 3. question1, 2(survey) 에 따른 wine_data 나누기

        if (question1_result, question2_result) == ('black', 'acid'):
            customer_data = customer_data[customer_data['type'] =='a']
        elif (question1_result, question2_result) == ('black', 'body'):
            customer_data = customer_data[customer_data['type'] =='b']
        elif (question1_result, question2_result) == ('milk_sweet', 'acid'):
            customer_data = customer_data[customer_data['type'] =='c']
        elif (question1_result, question2_result) == ('milk_sweet', 'body'):
            customer_data = customer_data[customer_data['type'] =='d']
        elif (question1_result, question2_result) == ('tea', 'acid'):
            customer_data = customer_data[customer_data['type'] =='e']
        elif (question1_result, question2_result) == ('tea', 'body'):
            customer_data = customer_data[customer_data['type'] =='f']

        # 4. question3(food)에 따른 데이터프레임 나누기

        if question3_result == 'meat':
            customer_data = customer_data[customer_data['meat'] == 1]
        elif question3_result == 'cheese':
            customer_data = customer_data[customer_data['cheese'] == 1]
        elif question3_result == 'seafood':
            customer_data = customer_data[customer_data['seafood'] == 1]
        elif question3_result == 'vegetables':
            customer_data = customer_data[customer_data['vegetables'] == 1]
        elif question3_result == 'dessert':
            customer_data = customer_data[customer_data['dessert'] == 1]

        # 5. question4(aroma)에 따른 데이터프레임 나누기

        if 'floral' in question4_result:
            customer_data = customer_data[customer_data['floral'] == 1]
        elif 'fruity' in question4_result:
            customer_data = customer_data[customer_data['fruity'] == 1]
        elif 'citrus' in question4_result:
            customer_data = customer_data[customer_data['citrus'] == 1]
        elif 'oriental_spice' in question4_result:
            customer_data = customer_data[customer_data['oriental_spice'] == 1]
        elif 'oriental_leather' in question4_result:
            customer_data = customer_data[customer_data['oriental_leather'] == 1]
        elif 'earth' in question4_result:
            customer_data = customer_data[customer_data['earth'] == 1]

        # 6. 투표 가장 많이 받은 아이템 sort

        customer_data_counts = customer_data['title'].value_counts()

        customer_data = pd.merge(customer_data, customer_data_counts, how='left', left_on ='title',
        right_on =customer_data_counts.index)

        customer_data.rename(columns = {'title_y' : 'counts'}, inplace=True)
        customer_data.rename(columns = {'title_x' : 'title'}, inplace=True)

        customer_data = customer_data.sort_values(by='counts', ascending=False)
        customer_data.drop_duplicates(keep='first', inplace=True)
        customer_data = customer_data.iloc[:,1:]

        # 7. 가격을 무시한 베스트 상품 찾기

        best_review = pd.DataFrame(columns=['title', 'alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                              'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                             'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                              'new_world', 'korea','price', 'link'])

        data =list(customer_data.iloc[0,:][['title','alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                              'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                             'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                              'new_world', 'korea', 'price', 'link']])

        data = pd.Series(data, index=best_review.columns)
        best_review = best_review.append(data, ignore_index=True)
        best_review.set_index('title', drop=False, inplace=True)

        # 8. price 적용한 데이터프레임

        if best_review['price'][0] > question5_answer:
            if len(customer_data[customer_data['price'] <= question5_answer]) >= 1:
                customer_data_price = customer_data[customer_data['price'] <= question5_answer]
                customer_data_price = customer_data_price[['title', 'alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                                      'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                                     'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                                      'new_world', 'korea']]
                best_review.drop(['price', 'link'], axis=1, inplace=True)
                best_review = best_review.append(customer_data_price)
                best_review.set_index('title', inplace=True)

                # 커피취향, 음식, 아로마, 가격대가 모두 맞고 베스트 상품과 유사도가 가장 높은 와인
                recommend_wine = cosine_similarity(best_review, best_review)
                recommend_wine_df = pd.DataFrame(data=recommend_wine, index = best_review.index, columns=best_review.index)
                recommend_wine_name = recommend_wine_df.iloc[0,:].sort_values(ascending=False)[1:2]
                for i in range(len(customer_data)):
                    if customer_data['title'].iloc[i] == recommend_wine_name.index:
                        result["link"] = customer_data['link'].iloc[i]
                        return result
                        break
            else:
                result["code"] = 201
                result["msg"] = "₩{} 가격대에는 적절한 와인이 없는 것 같아요 😭. 가끔은 평소와 다른 와인이 하루를 더 특별하게 만들어준답니다:)".format(question5_answer)
                return result

        elif best_review['price'][0] <= question5_answer:
            recommend_wine_df = best_review
            result["link"] = recommend_wine_df['link'][0]
            return result
    
    result = find_wine(int(request.args['Coffee_Preference']), 
                     int(request.args['Coffee_Bean']),
                     int(request.args['Food_Pairing']),
                     int(request.args['Aroma']), 
                     int(request.args['Price']), {"code": 200})
    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True)

Overwriting kakao/app.py


## HTML 작성

In [11]:
%%writefile kakao/templates/webservice.html
<!DOCTYPE html>
<html lang="en">
<head>
	<title>Wine Recommendation based on Coffee Preference</title>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<style type="text/css">
*{margin:0;padding:0;box-sizing:border-box}body,html{height:100%;font-family:Poppins-Regular,sans-serif}a{font-family:Poppins-Regular;font-size:14px;line-height:1.7;color:#666;margin:0;transition:all .4s;-webkit-transition:all .4s;-o-transition:all .4s;-moz-transition:all .4s}a:focus{outline:0!important}a:hover{text-decoration:none}h1,h2,h3,h4,h5,h6{margin:0}input{outline:0;border:none}input:focus::-webkit-input-placeholder{color:transparent}input:focus:-moz-placeholder{color:transparent}input:focus::-moz-placeholder{color:transparent}input:focus:-ms-input-placeholder{color:transparent}input::-webkit-input-placeholder{color:#adadad}input:-moz-placeholder{color:#adadad}input::-moz-placeholder{color:#adadad}input:-ms-input-placeholder{color:#adadad}button{outline:0!important;border:none;background:0 0}button:hover{cursor:pointer}.container{max-width:1200px}.container-contact100{width:100%;min-height:100vh;display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex;flex-wrap:wrap;justify-content:center;align-items:center;padding:15px;background:#a64bf4;background:-webkit-linear-gradient(45deg,#00dbde,#fc00ff);background:-o-linear-gradient(45deg,#00dbde,#fc00ff);background:-moz-linear-gradient(45deg,#00dbde,#fc00ff);background:linear-gradient(45deg,#00dbde,#fc00ff)}.wrap-contact100{width:500px;background:#fff;border-radius:10px;overflow:hidden;padding:42px 55px 45px 55px}.contact100-form{width:100%}.contact100-form-title{display:block;font-family:Poppins-Bold;font-size:39px;color:#333;line-height:1.2;text-align:center;padding-bottom:10px}.wrap-input100{width:100%;position:relative;border-bottom:2px solid #d9d9d9;padding-bottom:10px;margin-bottom:15px}.input100{display:block;width:100%;background:0 0;font-family:Poppins-Medium;font-size:18px;color:#333;line-height:1.2;padding:0 5px}.focus-input100{position:absolute;display:block;width:100%;height:100%;top:0;left:0;pointer-events:none}.focus-input100::before{content:"";display:block;position:absolute;bottom:-2px;left:0;width:0;height:2px;-webkit-transition:all .4s;-o-transition:all .4s;-moz-transition:all .4s;transition:all .4s;background:#7f7f7f}input.input100{height:40px}.input100:focus+.focus-input100::before{width:100%}.has-val.input100+.focus-input100::before{width:100%}.container-contact100-form-btn{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex;flex-wrap:wrap;justify-content:center;padding-top:13px}.wrap-contact100-form-btn{width:100%;display:block;position:relative;z-index:1;border-radius:25px;overflow:hidden;margin:0 auto}.contact100-form-bgbtn{position:absolute;z-index:-1;width:300%;height:100%;background:#a64bf4;background:-webkit-linear-gradient(left,#00dbde,#fc00ff,#00dbde,#fc00ff);background:-o-linear-gradient(left,#00dbde,#fc00ff,#00dbde,#fc00ff);background:-moz-linear-gradient(left,#00dbde,#fc00ff,#00dbde,#fc00ff);background:linear-gradient(left,#00dbde,#fc00ff,#00dbde,#fc00ff);top:0;left:-100%;-webkit-transition:all .4s;-o-transition:all .4s;-moz-transition:all .4s;transition:all .4s}.contact100-form-btn{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex;justify-content:center;align-items:center;padding:0 20px;width:100%;height:50px;font-family:Poppins-Medium;font-size:16px;color:#fff;line-height:1.2}.wrap-contact100-form-btn:hover .contact100-form-bgbtn{left:0}.contact100-form-btn i{-webkit-transition:all .4s;-o-transition:all .4s;-moz-transition:all .4s;transition:all .4s}.contact100-form-btn:hover i{-webkit-transform:translateX(10px);-moz-transform:translateX(10px);-ms-transform:translateX(10px);-o-transform:translateX(10px);transform:translateX(10px)}@media (max-width:576px){.wrap-contact100{padding:72px 15px 65px 15px}}.wrap-contact100{width:800px;background:#fff;border-radius:10px;overflow:hidden;padding:42px 55px 45px 55px}.contact100-form-title{font-size:26px}
</style>
	<div class="container-contact100">
		<div class="wrap-contact100">
			<form class="contact100-form validate-form">
				<span class="contact100-form-title" style="font-size:32px">당신의 커피 취향을 알려주세요.</span>
				
                <a id="output-a" href="#" target="_blink"><span class="contact100-form-title" id="output" style="color: #900C3F;" >&nbsp;</span></a>
                                                                                                          
                                                                                                          
				<div class="wrap-input100 " >
					<span class="label-input100">평소에 어떤 커피를 즐겨 마시나요?</span>
					<select  class="input100" id="Coffee_Preference" name="Coffee_Preference" style="padding: 7px 5px;margin-top: 10px;border: 2px solid #d9d9d9;" >
							<option value="">선호하는 커피를 골라주세요</option>
							<option value="0">아메리카노</option>
							<option value="1">달달한 라떼</option>
							<option value="2">차</option>
						</select>
					<span class="focus-input100"></span>
				</div>
				<div class="wrap-input100" style="border-bottom: 0px;" >
					<span class="label-input100">원두는 어떤 스타일을 선호하시나요?</span>
						<select  class="input100" id="Coffee_Bean" name="Coffee_Bean" style="padding: 10px 5px;margin-top: 15px;border: 2px solid #d9d9d9;" >
							<option value="">원두를 선택해주세요.</option>
							<option value="0">산미가 강한 타입</option>
							<option value="1">바디감이 있는 타입</option>
						</select>
					<span class="focus-input100"></span>
				</div>
				<div class="wrap-input100" style="border-bottom: 0px;" >
					<span class="label-input100">오늘은 어떤 음식과 마실건가요?</span>
						<select class="input100"  id="Food_Pairing" name="Food_Pairing" style="padding: 10px 5px;margin-top: 15px;border: 2px solid #d9d9d9;" >
							<option value="">음식을 선택해주세요.</option>
							<option value="0">육류</option>
							<option value="1">치즈</option>
							<option value="2">해산물</option>
							<option value="3">채소 혹은 과일</option>
							<option value="4">디저트</option>
						</select>
					<span class="focus-input100"></span>
				</div>

                <div class="wrap-input100" style="border-bottom: 0px;" >
					<span class="label-input100">어떤 향을 좋아하세요?</span>
						<select class="input100"  id="Aroma" name="Aroma" style="padding: 10px 5px;margin-top: 15px;border: 2px solid #d9d9d9;" >
                            <option value="">좋아하는 향을 알려주세요.</option>
							<option value="0">꽃향</option>
							<option value="1">과일향</option>
							<option value="2">시트러스향</option>
							<option value="3">스파이스향</option>
							<option value="4">스모키향</option>
							<option value="5">숲향</option>                                                                                                      
						</select>
					<span class="focus-input100"></span>
				</div>
                                                                                                                                                                                      
				<div class="wrap-input100 " >
					<span class="label-input100">꼭 비싸지 않아도 괜찮아요 :)</span>
					<input class="input100" type="text" id="Price" name="Price" placeholder="eg. 30000">
					<span class="focus-input100"></span>
				</div>                            
                                                                                                          
				<div class="container-contact100-form-btn">
					<div class="wrap-contact100-form-btn">
						<div class="contact100-form-bgbtn"></div>
						<button class="contact100-form-btn" id='submit' type="button">오늘 밤 좋은 시간 보내세요.</button>
					</div>
				</div>
			</form>
		</div>
	</div>
	<script type="text/javascript" src="http://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
	<script type="text/javascript">
        $(document).ready(function(){
            console.log("init jquery");
            $("#submit").on("click", function(){
                
                var url = "/predict";
                    url += "?Coffee_Preference=" + $('#Coffee_Preference').val();
                    url += "&Coffee_Bean=" + $('#Coffee_Bean').val();
                    url += "&Food_Pairing=" + $('#Food_Pairing').val();
                    url += "&Aroma=" + $('#Aroma').val();
                    url += "&Price=" + $('#Price').val();                
                $.getJSON(url, function(data){
                    console.log(data);
                    if(data.code == 201){
                        alert(data.msg);
                    }else{
                        $("#output").text(data.link);
                        $("#output-a").attr("href", data.link);
                        location.href = data.link;
                        
                    }
                    
                    
                })
            })
        })
	</script>
</body>
</html>

Overwriting kakao/templates/webservice.html


In [2]:
!python ./kakao/app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 129-147-749
127.0.0.1 - - [09/Nov/2020 13:17:25] "GET /main HTTP/1.0" 200 -
127.0.0.1 - - [09/Nov/2020 13:18:04] "GET /main HTTP/1.0" 200 -
127.0.0.1 - - [09/Nov/2020 13:19:08] "GET /main HTTP/1.0" 200 -
^C
